https://pypi.python.org/pypi/pyoctree

In [1]:
import numpy as np
import vtk

In [2]:
# Open mesh in ply format
def open_ply(filepath):
    import numpy
    f=open(filepath,'r');
    i=0;
    ip=0;
    it=0;
    np=0;
    nt=0;
    for str in f:
        i+=1;
        arr=str.split(" ");
        if(arr[0]=="element" and arr[1]=="vertex"):
            np=int(arr[2]);
            p=numpy.zeros((np,3));
        elif(arr[0]=="element" and arr[1]=="face"):
            nt=int(arr[2]);
            t=numpy.zeros((nt,3));
        elif(i>=11 and i<11+np):
            p[ip,0]=float(arr[0]);
            p[ip,1]=float(arr[1]);
            p[ip,2]=float(arr[2]);
            ip+=1;
        elif(i>=11+np and i<11+np+nt):
            t[it,0]=int(arr[1]);
            t[it,1]=int(arr[2]);
            t[it,2]=int(arr[3]);
            it+=1;
    mesh={};
    mesh["np"]=np;
    mesh["nt"]=nt;
    mesh["p"]=p;
    mesh["t"]=t;
    return mesh;

In [3]:
def findClosestNode(ep,tree):
    import scipy.spatial.distance as spdist    
    d_min = 100
    nid_min = -1
    #print 'ep',ep
    #for ep in endpoints:
    for j in range(len(tree.root.branches)):
        b = tree.root.branches[j]
        b_pos = b.position
        #print b_pos
        d = spdist.euclidean(b_pos,ep)
        #print d
        if d<=d_min :
            d_min = d
            nid_min = b.nid
    #print d_min,nid_min

    node_min = tree.getNodeFromId(nid_min)
    node_curr = node_min
    d_curr = 100
    nid_curr = -1
    while(not node_min.isLeaf):
        for j in node_min.branches:
            #b = tree.root.branches[j]
            b_pos = j.position
            #print b_pos
            d = spdist.euclidean(b_pos,ep)
            #print 'newly computed distance',d
            if (d<=d_curr) and not(j.isLeaf and not j.polyList) :
                #print 'new shorter distance than',d_curr
                d_curr = d
                nid_curr = j.nid
                node_curr = tree.getNodeFromId(nid_curr)
        #print 'current shortest distance and its node',d_curr,node_curr
        if (not node_min.polyList):
            #print 'no poly'
            d_min = d_curr
            nid_min = nid_curr
            node_min = node_curr
            d_curr = 100
            nid_curr = -1
        elif (node_curr == node_min) and (node_min.polyList):
            #print 'equal & poly'
            break
    
    #print d_min,nid_min
    return d_min,nid_min


In [4]:
# Normal of triangle a,b,c
def normal(a,b,c):
    v=np.cross(b-a,c-a)
    n=v/np.linalg.norm(v)
    return n;

In [41]:
def findClosestPoint(ep, mesh):
    import scipy.spatial.distance as spdist
#read mesh
    npt=mesh["np"];
    nt=mesh["nt"];
    pt=mesh["p"];
    t=mesh["t"];
    
#create tree
    t=t.astype(dtype=np.int32)

    from pyoctree import pyoctree as ot
    tree = ot.PyOctree(pt,t)

# Print out basic Octree data
    #print "Size of Octree               = %.3fmm" % tree.root.size
    #print "Number of Octnodes in Octree = %d" % tree.getNumberOfNodes()
    #print "Number of polys in Octree    = %d" % tree.numPolys
    
#find clostest node and the triangles in this node
    d_node,node_id = findClosestNode(ep,tree)
    t_index = tree.getNodeFromId(node_id).polyList

#look for the closest node within the preselected list of vertices
    n=np.zeros((len(t_index),3));

    # same normal for all the points of a triangle, 
    #no influence of the neighboring triangle 
    #and could be duplicated points with different normals

    for j in range(len(t_index)):
        i = t_index[j]
        nn=normal(pt[t[i,0]],pt[t[i,1]],pt[t[i,2]]);
        n[j,0]=nn[0]; 
        n[j,1]=nn[1];
        n[j,2]=nn[2];

    d = 1000000

    for i in range(len(t_index)):
        for k in range(3):
            v = ep - pt[t[i,k]] #distance to the normal of the triangle
            d1 = np.absolute(np.dot(v, n[i]))

            #d1 = spdist.euclidean(ep,pt[t[i,k]]) #euclidean distance point/endpoint

            if d1 < d:
                d = d1
                cp = pt[i]
                point = i
    #print d,point
    return d, point

In [6]:
pathDesktop = 'C:/Users/cdelettre/Desktop/'

subj = 'P8_F10'
path = 'C:/Users/cdelettre/Desktop/Ferrets/'+subj+'/'
fmesh = path+subj+'_mesh.ply'
mesh = open_ply(fmesh)
#npt=mesh["np"];
#nt=mesh["nt"];
#p=mesh["p"];
#t=mesh["t"];

In [7]:
fendpoints = path+'P8_F10_endpoints_adjusted.txt'
endpoints = np.loadtxt(fendpoints)

In [42]:
import time

ep_try = endpoints[0:10]
printInfos = True

ts = time.time()
#print 'ts', ts

#ep = endpoints[22]
for ep in ep_try:
    
    d_point,point = findClosestPoint(ep,mesh)

    if (printInfos):
        print 'point coordinates:',ep
        #print 'node id:',node_id
        #print 'nb of vertices to be checked:',len(t_index)
        #print 'node distance:',d_node
        print 'point distance:',d_point,'point id:',point
        print '\n'
        
        
te = time.time()
#print 'te',te
print 'duration',te-ts

point coordinates: [ 12.60884   13.26452    3.235695]
point distance: 2.80029378299 point id: 97


point coordinates: [  9.88554   13.01362    2.227335]
point distance: 1.40312131593 point id: 95


point coordinates: [  7.44064   11.59352    3.015165]
point distance: 4.46259202296 point id: 75


point coordinates: [  4.70904   10.77582    1.958195]
point distance: 5.65998756573 point id: 71


point coordinates: [  7.83584   12.71982    4.141665]
point distance: 3.76215469847 point id: 95


point coordinates: [  5.45824   11.26372    1.902015]
point distance: 4.68225859818 point id: 95


point coordinates: [  2.38354   10.00502    2.503095]
point distance: 7.46650381225 point id: 61


point coordinates: [ 5.19964   9.72072   2.121925]
point distance: 2.66300715332 point id: 170


point coordinates: [ 20.32944    8.68462    1.865145]
point distance: 8.1221598744 point id: 73


point coordinates: [ 18.26274    8.55192    2.202045]
point distance: 6.10564966652 point id: 73


duration 2.25

Annex

In [8]:
t=t.astype(dtype=np.int32)

from pyoctree import pyoctree as ot
tree = ot.PyOctree(p,t)

# Print out basic Octree data
print "Size of Octree               = %.3fmm" % tree.root.size
print "Number of Octnodes in Octree = %d" % tree.getNumberOfNodes()
print "Number of polys in Octree    = %d" % tree.numPolys

tree.getOctreeRep()